In [1]:
import torch
from collections import defaultdict
import numpy as np
from utils import metrics

In [2]:
def model_pred(model, testdata, K, test_num_neg=99):
	model.eval()
	model = model.cpu()

	ndcg = np.zeros(K)
	recall_matrix = np.zeros(len(cut_offs))
	map_matrix = np.zeros(len(cut_offs))
	auc_value = 0
	mrr_value = 0

	with torch.no_grad():
		for u in testdata.keys():
			items = testdata[u]
			label = np.array([1] + [0] * test_num_neg)
			pred = model(torch.tensor(u), torch.tensor(items))
			recall, ap, mrr, auc = metrics.map_recall_at_k(label, pred.detach().numpy(), cut_offs)
			recall_matrix += recall
			map_matrix += ap
			mrr_value += mrr
			ndcg += metrics.normalized_discounted_cumulative_gain_matrix(K, label, pred.detach().numpy())
			auc_value += auc
	user_count = len(testdata)
	print('test ndcg = ', ndcg / user_count)
	print('test recall = ', recall_matrix / user_count)
	print('test map = ', map_matrix / user_count)
	print('test AUC = ', auc_value / user_count)
	print('test MRR = ', mrr_value / user_count)

In [3]:
if __name__ == '__main__':
	testpath = 'E:/Lab/code_data/data/citeulike-a-master/processed_data/citeulike_test.csv'
	K = 10
	cut_offs = [2,4,6,8,10,15,20]

	print('--------------load test data--------------')
	test_data = defaultdict()
	with open(testpath, 'r') as fd:
		line = fd.readline()
		while line != None and line != '':
			arr = line.split(',')
			u = int(arr[0])
			items = []
			for i in arr[1:]:
				items.append(int(i))
			test_data[u] = items
			line = fd.readline()

	print('--------------test--------------')
	modelpath = '../save_model/citeulike/CiteULike_DLPR_embsize_8_K_10_zetastep_0.2_maxiter_10_tau_10.pkl'
	print('------------------load model------------------')
	model = torch.load(modelpath, map_location=torch.device('cpu'))
	print(model)
	model_pred(model, test_data, K)
	print('=='*30)

--------------load test data--------------
--------------test--------------
------------------load model------------------


E:\ProgramData\Miniconda3\lib\site-packages\torch\serialization.py:671: SourceChangeWarning: source code of class 'utils.model.MF' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
E:\ProgramData\Miniconda3\lib\site-packages\torch\serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


MF(
  (user_emb): Embedding(5551, 8)
  (item_emb): Embedding(16980, 8)
)
test ndcg =  [0.14555361 0.18287656 0.20194642 0.21351158 0.22307187 0.23069646
 0.23620985 0.24081384 0.24426991 0.24803861]
test recall =  [0.20470909 0.26970228 0.3158202  0.34695466 0.37147305 0.44872543
 0.51488616]
test map =  [0.17513135 0.19455796 0.20306804 0.20725521 0.20983461 0.21580968
 0.21954434]
test AUC =  0.7425451243314667
test MRR =  0.23171079239197856
